# HW 3: Neural Machine Translation

In this homework you will build a full neural machine translation system using an attention-based encoder-decoder network to translate from German to English. The encoder-decoder network with attention forms the backbone of many current text generation systems. See [Neural Machine Translation and Sequence-to-sequence Models: A Tutorial](https://arxiv.org/pdf/1703.01619.pdf) for an excellent tutorial that also contains many modern advances.

## Goals


1. Build a non-attentional baseline model (pure seq2seq as in [ref](https://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf)). 
2. Incorporate attention into the baseline model ([ref](https://arxiv.org/abs/1409.0473) but with dot-product attention as in class notes).
3. Implement beam search: review/tutorial [here](http://www.phontron.com/slides/nlp-programming-en-13-search.pdf)
4. Visualize the attention distribution for a few examples. 

Consult the papers provided for hyperparameters, and the course notes for formal definitions.

This will be the most time-consuming assignment in terms of difficulty/training time, so we recommend that you get started early!

## Setup

This notebook provides a working definition of the setup of the problem itself. Feel free to construct your models inline, or use an external setup (preferred) to build your system.

In [ ]:
# Text text processing library and methods for pretrained word embeddings
from torchtext import data
from torchtext import datasets

We first need to process the raw data using a tokenizer. We are going to be using spacy, which can be installed via:  
  `[sudo] pip install spacy`  
  
Tokenizers for English/German can be installed via:  
  `[sudo] python -m spacy download en`  
  `[sudo] python -m spacy download de`
  
This isn't *strictly* necessary, and you can use your own tokenization rules if you prefer (e.g. a simple `split()` in addition to some rules to acccount for punctuation), but we recommend sticking to the above.

In [ ]:
import spacy
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]


Note that we need to add the beginning-of-sentence token `<s>` and the end-of-sentence token `</s>` to the 
target so we know when to begin/end translating. We do not need to do this on the source side.

In [ ]:
BOS_WORD = '<s>'
EOS_WORD = '</s>'
DE = data.Field(tokenize=tokenize_de)
EN = data.Field(tokenize=tokenize_en, init_token = BOS_WORD, eos_token = EOS_WORD) # only target needs BOS/EOS

Let's download the data. This may take a few minutes.

**While this dataset of 200K sentence pairs is relatively small compared to others, it will still take some time to train. So we are going to be only working with sentences of length at most 20 for this homework. Please train only on this reduced dataset for this homework.**

In [ ]:
MAX_LEN = 20
train, val, test = datasets.IWSLT.splits(exts=('.de', '.en'), fields=(DE, EN), 
                                         filter_pred=lambda x: len(vars(x)['src']) <= MAX_LEN and 
                                         len(vars(x)['trg']) <= MAX_LEN)
print(train.fields)
print(len(train))
print(vars(train[0]))

Now we build the vocabulary and convert the text corpus into indices. We are going to be replacing tokens that occurred less than 5 times with `<unk>` tokens, and take the rest as our vocab.

In [ ]:
MIN_FREQ = 5
DE.build_vocab(train.src, min_freq=MIN_FREQ)
EN.build_vocab(train.trg, min_freq=MIN_FREQ)
print(DE.vocab.freqs.most_common(10))
print("Size of German vocab", len(DE.vocab))
print(EN.vocab.freqs.most_common(10))
print("Size of English vocab", len(EN.vocab))
print(EN.vocab.stoi["<s>"], EN.vocab.stoi["</s>"]) #vocab index for <s>, </s>

Now we split our data into batches as usual. Batching for MT is slightly tricky because source/target will be of different lengths. Fortunately, `torchtext` lets you do this by allowing you to pass in a `sort_key` function. This will minimizing the amount of padding on the source side, but since there is still some padding you will inadvertendly "attend" to these padding tokens. 

One way to get rid of padding is to pass a binary `mask` vector to your attention module so its attention score (before the softmax) is minus infinity for the padding token. Another way (which is how we do it for our projects, e.g. opennmt) is to manually sort data into batches so that each batch has exactly the same source length (this means that some batches will be less than the desired batch size, though).

However, for this homework padding won't matter too much, so it's fine to ignore it.

In [ ]:
BATCH_SIZE = 32
train_iter, val_iter = data.BucketIterator.splits((train, val), batch_size=BATCH_SIZE, device=0,
                                                  repeat=False, sort_key=lambda x: len(x.src))

Let's check to see that the BOS/EOS token is indeed appended to the target (English) sentence.

In [ ]:
batch = next(iter(val_iter))
print("Source")
print(batch.src)
print("Target")
print(batch.trg)
print(batch.src.volatile)

Success! Now that we've processed the data, we are ready to begin modeling.

# Baseline Model

In [1]:
import torch
import random
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torchtext import data
from torchtext import datasets
import spacy

In [2]:
use_cuda = torch.cuda.is_available()
print(use_cuda)

False


In [3]:
# Set up 
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

BOS_WORD = '<s>'
EOS_WORD = '</s>'
DE = data.Field(tokenize=tokenize_de, init_token = BOS_WORD, eos_token = EOS_WORD)
EN = data.Field(tokenize=tokenize_en, init_token = BOS_WORD, eos_token = EOS_WORD) # only target needs BOS/EOS

MAX_LEN = 20
train, val, test = datasets.IWSLT.splits(exts=('.de', '.en'), fields=(DE, EN), 
                                         filter_pred=lambda x: len(vars(x)['src']) <= MAX_LEN and 
                                         len(vars(x)['trg']) <= MAX_LEN)
print(train.fields)
print(len(train))
print(vars(train[0]))

MIN_FREQ = 5
DE.build_vocab(train.src, min_freq=MIN_FREQ)
EN.build_vocab(train.trg, min_freq=MIN_FREQ)
print(DE.vocab.freqs.most_common(10))
print("Size of German vocab", len(DE.vocab))
print(EN.vocab.freqs.most_common(10))
print("Size of English vocab", len(EN.vocab))
print(DE.vocab.stoi["<s>"], DE.vocab.stoi["</s>"]) #vocab index for <s>, </s>
print(EN.vocab.stoi["<s>"], EN.vocab.stoi["</s>"]) #vocab index for <s>, </s>

{'src': <torchtext.data.field.Field object at 0x11b8f6438>, 'trg': <torchtext.data.field.Field object at 0x11e3220f0>}
119076
{'src': ['David', 'Gallo', ':', 'Das', 'ist', 'Bill', 'Lange', '.', 'Ich', 'bin', 'Dave', 'Gallo', '.'], 'trg': ['David', 'Gallo', ':', 'This', 'is', 'Bill', 'Lange', '.', 'I', "'m", 'Dave', 'Gallo', '.']}
[('.', 113253), (',', 67237), ('ist', 24189), ('die', 23778), ('das', 17102), ('der', 15727), ('und', 15622), ('Sie', 15085), ('es', 13197), ('ich', 12946)]
Size of German vocab 13355
[('.', 113433), (',', 59512), ('the', 46029), ('to', 29177), ('a', 27548), ('of', 26794), ('I', 24887), ('is', 21775), ("'s", 20630), ('that', 19814)]
Size of English vocab 11560
2 3
2 3


In [13]:
BATCH_SIZE = 32
if use_cuda: 
    train_iter, val_iter = data.BucketIterator.splits((train, val), batch_size=BATCH_SIZE, device=0,
                                                  repeat=False, sort_key=lambda x: len(x.src))
else: 
    train_iter, val_iter = data.BucketIterator.splits((train, val), batch_size=BATCH_SIZE, device=-1,
                                                  repeat=False, sort_key=lambda x: len(x.src)) 

In [55]:
for batch in iter(train_iter): 
    print("Source")
    print(len(batch.src))
#     print("Target")
#     print(len(batch.trg))

Source
11
Source
15
Source
10
Source
9
Source
20
Source
11
Source
10
Source
16
Source
15
Source
15
Source
17
Source
11
Source
9
Source
8
Source
13
Source
16
Source
11
Source
15
Source
11
Source
22
Source
14
Source
17
Source
17
Source
19
Source
18
Source
7
Source
21
Source
21
Source
12
Source
9
Source
11
Source
16
Source
14
Source
6
Source
10
Source
9
Source
11
Source
18
Source
15
Source
8
Source
12
Source
20
Source
15
Source
13
Source
18
Source
12
Source
20
Source
9
Source
14
Source
10
Source
10
Source
16
Source
8
Source
15
Source
8
Source
8
Source
17
Source
10
Source
20
Source
13
Source
19
Source
6
Source
14
Source
16
Source
14
Source
19
Source
10
Source
9
Source
16
Source
7
Source
22
Source
7
Source
7
Source
10
Source
13
Source
5
Source
13
Source
12
Source
11
Source
10
Source
7
Source
8
Source
16
Source
11
Source
8
Source
9
Source
14
Source
13
Source
17
Source
12
Source
13
Source
17
Source
14
Source
12
Source
19
Source
18
Source
12
Source
18
Source
18
Source
12
Source
12
Source
9
Sou

13
Source
17
Source
15
Source
11
Source
7
Source
8
Source
16
Source
18
Source
16
Source
13
Source
20
Source
15
Source
6
Source
10
Source
20
Source
8
Source
11
Source
7
Source
4
Source
11
Source
10
Source
16
Source
14
Source
9
Source
15
Source
14
Source
19
Source
12
Source
17
Source
9
Source
14
Source
21
Source
13
Source
7
Source
22
Source
16
Source
15
Source
18
Source
22
Source
17
Source
10
Source
13
Source
8
Source
21
Source
13
Source
10
Source
6
Source
10
Source
5
Source
19
Source
12
Source
16
Source
12
Source
19
Source
13
Source
8
Source
15
Source
14
Source
9
Source
9
Source
14
Source
8
Source
14
Source
20
Source
17
Source
14
Source
21
Source
7
Source
14
Source
9
Source
15
Source
19
Source
14
Source
14
Source
11
Source
14
Source
17
Source
20
Source
10
Source
15
Source
13
Source
18
Source
6
Source
6
Source
13
Source
12
Source
11
Source
18
Source
11
Source
9
Source
16
Source
17
Source
13
Source
22
Source
15
Source
22
Source
19
Source
20
Source
20
Source
8
Source
14
Source
20
Source
11

Source
8
Source
11
Source
15
Source
11
Source
16
Source
11
Source
7
Source
17
Source
10
Source
13
Source
19
Source
11
Source
10
Source
14
Source
17
Source
5
Source
9
Source
11
Source
10
Source
14
Source
19
Source
14
Source
7
Source
9
Source
17
Source
8
Source
11
Source
9
Source
8
Source
6
Source
14
Source
12
Source
8
Source
16
Source
9
Source
12
Source
15
Source
13
Source
10
Source
22
Source
19
Source
18
Source
5
Source
10
Source
12
Source
13
Source
18
Source
9
Source
12
Source
16
Source
9
Source
18
Source
9
Source
8
Source
15
Source
13
Source
18
Source
16
Source
8
Source
20
Source
22
Source
15
Source
20
Source
12
Source
13
Source
9
Source
22
Source
12
Source
17
Source
21
Source
10
Source
18
Source
13
Source
11
Source
13
Source
6
Source
14
Source
12
Source
15
Source
20
Source
12
Source
20
Source
11
Source
14
Source
15
Source
14
Source
16
Source
9
Source
13
Source
16
Source
11
Source
18
Source
17
Source
9
Source
11
Source
21
Source
10
Source
6
Source
16
Source
10
Source
8
Source
10
Sour

Source
8
Source
13
Source
18
Source
13
Source
22
Source
15
Source
9
Source
15
Source
18
Source
12
Source
15
Source
9
Source
14
Source
12
Source
22
Source
7
Source
18
Source
15
Source
8
Source
7
Source
12
Source
14
Source
11
Source
7
Source
10
Source
14
Source
17
Source
8
Source
13
Source
20
Source
5
Source
10
Source
13
Source
17
Source
19
Source
18
Source
9
Source
19
Source
8
Source
15
Source
15
Source
12
Source
21
Source
17
Source
14
Source
16
Source
9
Source
20
Source
12
Source
10
Source
10
Source
12
Source
17
Source
11
Source
14
Source
10
Source
11
Source
11
Source
11
Source
11
Source
11
Source
10
Source
16
Source
20
Source
6
Source
14
Source
17
Source
16
Source
9
Source
16
Source
16
Source
13
Source
19
Source
16
Source
22
Source
20
Source
18
Source
11
Source
17
Source
17
Source
9
Source
14
Source
8
Source
10
Source
13
Source
22
Source
15
Source
7
Source
10
Source
16
Source
8
Source
11
Source
12
Source
12
Source
9
Source
14


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration
  """Entry point for launching an IPython kernel.


In [76]:
SOS_token = 2
EOS_token = 3
teacher_forcing_ratio = 0.5
class EncoderLSTM(nn.Module):
    def __init__(self, input_size, h_size, batch_size, n_layers=1, dropout=0):
        super(EncoderLSTM, self).__init__()
        self.num_layers = n_layers
        self.hidden_size = h_size
        self.batch_size = batch_size
        self.embed = nn.Embedding(input_size, h_size)
        self.lstm = nn.LSTM(h_size, h_size, dropout=dropout, num_layers=n_layers)

    def forward(self, input_src, hidden):
        embedded = self.embed(input_src)
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden

    def init_hidden(self):
        result = (Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_size)),
                  Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_size)))
        if use_cuda:
            return (Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_size)).cuda(),
                    Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_size)).cuda())
        else:
            return result
        
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, batch_size, dropout=0.1, n_layers=1, max_length=MAX_LEN + 2 ):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout = dropout
        self.max_length = max_length
        self.num_layers = n_layers
        self.batch_size = batch_size

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        
        self.dropout = nn.Dropout(self.dropout)
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input_data, hidden, encoder_outputs):
        #seq_len x batch_size x hidden_dim 
        embedded = self.embedding(input_data)
        embedded = self.dropout(embedded)
        
        #embedded[0] is batch_size x hidden_dim 
        #hidden[0] is batch_size x hidden_dim 
        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0][0]), 1)), dim=1)
        
        #atten_weights is batch_size x max_len      
        attn_applied = torch.bmm(attn_weights.unsqueeze(1),
                                 encoder_outputs.permute(1, 0, 2))
        #attn_applied is 1 x batch_size x hidden_dim
        
        output = torch.cat((embedded[0], attn_applied.squeeze(1)), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.lstm(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def init_hidden(self):
        result = (Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_size)),
                  Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_size)))
        if use_cuda:
            return (Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_size)).cuda(),
                    Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_size)).cuda())
        else:
            return result

def train_batch(input_variable, target_variable, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LEN+ 2 ):
    encoder_hidden = encoder.init_hidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    input_length = input_variable.size()[0]
    target_length = target_variable.size()[0]
    batch_size = target_variable.size()[1]
    loss = 0
    if batch_size != 32:
        return loss
    
    encoder_output_short, encoder_hidden = encoder(input_variable, encoder_hidden)
    
    encoder_outputs = Variable(torch.zeros(max_length, batch_size, encoder.hidden_size))
    encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs
    
    encoder_outputs[:input_length, :, :] = encoder_output_short 
    
    decoder_input = Variable(torch.ones(1, batch_size).long()) * SOS_token
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for t in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_input = target_variable[t].view(1, -1)
            decoder_input = decoder_input.cuda() if use_cuda else decoder_input
            loss += criterion(decoder_output.squeeze(0), target_variable[t]) / target_length
    else:
        for t in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = torch.max(decoder_output, 1)
            decoder_input = topi.view(1, -1)
            decoder_input = decoder_input.cuda() if use_cuda else decoder_input
            loss += criterion(decoder_output.squeeze(0), target_variable[t]) / target_length

    loss.backward()

    torch.nn.utils.clip_grad_norm(encoder.parameters(), 5.0)
    torch.nn.utils.clip_grad_norm(decoder.parameters(), 5.0)

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.data[0]


def validate(encoder, decoder, val_iter, criterion, max_length = MAX_LEN+2):
    total_loss = 0
    num_batches = 0
    for batch in iter(val_iter):
        
        input_length = batch.src.size()[0]
        target_length = batch.trg.size()[0]
        batch_size = batch.src.size()[1]
        if batch_size != 32:
            break
        encoder_hidden = encoder.init_hidden()
        
        encoder_output_short, encoder_hidden = encoder(batch.src, encoder_hidden)
    
        encoder_outputs = Variable(torch.zeros(max_length, batch_size, encoder.hidden_size))
        encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs

        encoder_outputs[:input_length, :, :] = encoder_output_short      
        
        
        decoder_input = Variable(torch.ones(1, batch_size).long() * SOS_token)
        decoder_input = decoder_input.cuda() if use_cuda else decoder_input

        decoder_hidden = encoder_hidden

        decoded_words = []
        loss = 0
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs) 
            topv, topi = torch.max(decoder_output, 1)
            decoded_words.append(topi)
            decoder_input = topi.view(1, -1)
            decoder_input = decoder_input.cuda() if use_cuda else decoder_input
            loss += criterion(decoder_output.squeeze(0), batch.trg[di])

        total_loss += loss.data[0] / target_length
        num_batches += 1
    return total_loss / num_batches


def trainIters(encoder, decoder, training_iter, valid_iter, learning_rate=0.7, num_epochs=20):
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss(ignore_index=1)
    
    for e in range(num_epochs):
        batch_len = 0
        total_loss = 0
        for batch in iter(training_iter):
            loss = train_batch(batch.src, batch.trg, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
            total_loss += loss
            batch_len += 1
        train_loss = total_loss / batch_len
        print("train loss: ", train_loss)
        print("train ppl: ", np.exp(train_loss))
        val_loss = validate(encoder, decoder, valid_iter, criterion)
        print("val loss: ", val_loss)
        print("val ppl: ", np.exp(val_loss))
#         torch.save(encoder.state_dict(), 'attn_encoder_model.pt')
#         torch.save(decoder.state_dict(), 'attn_decoder_model.pt')

In [75]:
hidden_size = 256
encoder1 = EncoderLSTM(len(DE.vocab), hidden_size, batch_size=32, dropout=0.3, n_layers=1)
decoder1 = AttnDecoderRNN(hidden_size, len(EN.vocab), batch_size=32, dropout=0.1, n_layers=1)
if use_cuda:
    encoder1 = encoder1.cuda()
    decoder1 = decoder1.cuda()
trainIters(encoder1, decoder1, list(train_iter)[:100], val_iter, num_epochs=5)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration
  import sys


train loss:  6.159294013977051
val loss:  5.715936188678063
train loss:  5.0293380379676815
val loss:  5.027225275351566
train loss:  4.769140453338623
val loss:  4.6651717657567175
train loss:  4.569118583202362
val loss:  4.850615899361763
train loss:  4.485138152837753
val loss:  5.090717225126644


In [ ]:
def evaluate(encoder, decoder, batch, max_length=MAX_LEN):

    target_length = batch.trg.size()[0]
    batch_length = batch.src.size()[1]
    
    encoder_hidden = encoder.initHidden()
    _, encoder_hidden = encoder(batch.src, encoder_hidden)
    decoder_input = Variable(torch.ones(1, batch_length).long()*SOS_token)
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input
    decoder_hidden = encoder_hidden


    decoded_words = []

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        topv, topi = torch.max(decoder_output, 2)
        decoded_words.append(topi)
        decoder_input = topi
        decoder_input = decoder_input.cuda() if use_cuda else decoder_input
            
    return torch.cat(decoded_words)

batch = list(iter(val_iter))[8]
# print("Source")
# print(batch.src)
# print("Target")
# print(batch.trg)
evaluate(encoder1, decoder1, batch)

## Assignment

Now it is your turn to build the models described at the top of the assignment. 

When a model is trained, use the following test function to produce predictions, and then upload to the kaggle competition: https://www.kaggle.com/c/cs287-hw3-s18/

For the final Kaggle test, we will provide the source sentence, and you are to predict the **first three words of the target sentence**. The source sentence can be found under `source_test.txt`

In [ ]:
!head source_test.txt

Similar to HW1, you are to predict the 100 most probable 3-gram that will begin the target sentence. The submission format will be as follows, where each word in the 3-gram will be separated by "|", and each 3-gram will be separated by space. For example, here is what an example submission might look like with 5 most-likely 3-grams (instead of 100).

```
id,word
1,Newspapers|talk|about When|I|was Researchers|call|the Twentysomethings|like|Alex But|before|long
2,That|'s|what Newspapers|talk|about You|have|robbed It|'s|realizing My|parents|wanted
3,We|forget|how We|think|about Proust|actually|links Does|any|other This|is|something
4,But|what|do And|it|'s They|'re|on My|name|is It|only|happens
```

When you print out your data, you will need to escape quotes and commas with the following command so that Kaggle does not complain. 

In [ ]:
def escape(l):
    return l.replace("\"", "<quote>").replace(",", "<comma>")

You should perform your hyperparameter search/early stopping/write-up based on perplexity, not the above metric. (In practice, people use a metric called [BLEU](https://www.aclweb.org/anthology/P02-1040.pdf), which is roughly a geometric average of 1-gram, 2-gram, 3-gram, 4-gram precision, with a brevity penalty for producing translations that are too short.)

Finally, as always please put up a (short) write-up following the template provided in the repository:  https://github.com/harvard-ml-courses/cs287-s18/blob/master/template/
